Data Set

Spotify Tracks Dataset

[Link Kaggle](https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 68.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=0357e635e024d4134c6d3b7742dec9cd16fbe0f3ec777e50e79ef01294575eea
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark

In [ ]:
sc = spark.sparkContext

In [ ]:
data = "/content/drive/MyDrive/data_sample/BigData2022/Spotify Tracks Dataset.csv"

In [ ]:
#to to read CSV file into RDD of Strings
lines = sc.textFile(data)

In [ ]:
print(type(lines))

<class 'pyspark.rdd.RDD'>


In [ ]:
lines.count()

114001

In [ ]:
lines.take(2)

[',track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre',
 '0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,FALSE,0.676,0.461,1,-6.746,0,0.143,0.0322,1.01E-06,0.358,0.715,87.917,4,acoustic']

In [ ]:
header_list = lines.take(1)[0].split(",")

In [ ]:
print(header_list)

['', 'track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre']


In [ ]:
len(header_list)

21

In [ ]:
def list_to_dict(l):
  dic = {}
  for i in range(len(l)):
    dic[l[i]] = i
  return(dic)

In [ ]:
header_dict = list_to_dict(header_list)

In [ ]:
print(header_dict)

{'': 0, 'track_id': 1, 'artists': 2, 'album_name': 3, 'track_name': 4, 'popularity': 5, 'duration_ms': 6, 'explicit': 7, 'danceability': 8, 'energy': 9, 'key': 10, 'loudness': 11, 'mode': 12, 'speechiness': 13, 'acousticness': 14, 'instrumentalness': 15, 'liveness': 16, 'valence': 17, 'tempo': 18, 'time_signature': 19, 'track_genre': 20}


In [ ]:
#remove header from rdd
header = lines.first()
lines_data = lines.filter(lambda line: line != header)

In [ ]:
#cheking that we have rdd without header
print(lines_data.take(1))

['0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,FALSE,0.676,0.461,1,-6.746,0,0.143,0.0322,1.01E-06,0.358,0.715,87.917,4,acoustic']


In [ ]:
#cheking number of rows in ddr without header
lines_data.count()

114000

For parsing CSV we need to map for every items in columns number of columns. 

It will be pairs of key values (number of column, item)

For this reason we create function that get rdd line as parameter and return list pairs of key values

In [ ]:
#func for creating list of pair(key,value)
def list_of_tuple (line_rdd):
  l = line_rdd.split(',') #split rdd line (string type)
  list_tup = []
  for i in range(len(l)): # number of columns (starting from 0 )
    list_tup.append((i,l[i]))
  return(list_tup)

In [ ]:
#maping function list_of_tuple on all rdd 
pairs_rdd = lines_data.flatMap(lambda line: list_of_tuple(line))

In [ ]:
#cheking type
type(pairs_rdd)

pyspark.rdd.PipelinedRDD

In [ ]:
#cheking result of mapping 
print(pairs_rdd.take(42))

[(0, '0'), (1, '5SuOikwiRyPMVoIQDJUgSV'), (2, 'Gen Hoshino'), (3, 'Comedy'), (4, 'Comedy'), (5, '73'), (6, '230666'), (7, 'FALSE'), (8, '0.676'), (9, '0.461'), (10, '1'), (11, '-6.746'), (12, '0'), (13, '0.143'), (14, '0.0322'), (15, '1.01E-06'), (16, '0.358'), (17, '0.715'), (18, '87.917'), (19, '4'), (20, 'acoustic'), (0, '1'), (1, '4qPNDBW1i3p13qLCt0Ki3A'), (2, 'Ben Woodward'), (3, 'Ghost (Acoustic)'), (4, 'Ghost - Acoustic'), (5, '55'), (6, '149610'), (7, 'FALSE'), (8, '0.42'), (9, '0.166'), (10, '1'), (11, '-17.235'), (12, '1'), (13, '0.0763'), (14, '0.924'), (15, '5.56E-06'), (16, '0.101'), (17, '0.267'), (18, '77.489'), (19, '4'), (20, 'acoustic')]


In [ ]:
#saving result of mapping
pairs_rdd.saveAsTextFile("pairs_rdd5.txt")

In [ ]:
#filter first 2 fields
filter_rdd = pairs_rdd.filter(lambda x: x[0]<2)

In [ ]:
filter_rdd.take(10)

[(0, '0'),
 (1, '5SuOikwiRyPMVoIQDJUgSV'),
 (0, '1'),
 (1, '4qPNDBW1i3p13qLCt0Ki3A'),
 (0, '2'),
 (1, '1iJBSr7s7jYXzM8EGcbK5b'),
 (0, '3'),
 (1, '6lfxq3CG4xtTiEg7opyCyx'),
 (0, '4'),
 (1, '5vjLSffimiIP26QG5WcN2K')]

In [ ]:
#saving result of filter - first 2 fields
filter_rdd.saveAsTextFile("filter_rdd.txt")

In [ ]:
#groupping by number of columns (by key)
rdd_groupping = pairs_rdd.groupByKey().mapValues(list)

In [ ]:
#saving reault of grouping 
rdd_groupping.saveAsTextFile("rdd_groupping.txt")

In [ ]:
#now we can filter rdd take first 4 columns and group by key(number of columns)
rdd_groupping_and_filter = pairs_rdd.filter(lambda x: x[0]<4).groupByKey().mapValues(list)

In [ ]:
#saving reault of grouping filterd rdd
rdd_groupping_and_filter.saveAsTextFile("rdd_groupping_and_filter2.txt")

In [ ]:
#chek how many groups we get(should be 4 = 4 columns)
rdd_groupping_and_filter.count()

4

In [ ]:
#count how many items has each group
rdd_groupping_and_filter2 = pairs_rdd.groupByKey().mapValues(len)

In [ ]:
rdd_groupping_and_filter2.take(21)

[(0, 114000),
 (1, 114000),
 (2, 114000),
 (3, 114000),
 (4, 114000),
 (5, 114000),
 (6, 114000),
 (7, 114000),
 (8, 114000),
 (9, 114000),
 (10, 114000),
 (11, 114000),
 (12, 114000),
 (13, 114000),
 (14, 114000),
 (15, 114000),
 (16, 114000),
 (17, 114000),
 (18, 114000),
 (19, 114000),
 (20, 114000)]

Some more filters and grouping

In [ ]:
#we also can filter value by some condition: track_genre = afrobeat 
filter_by_value = lines_data.filter(lambda a: a.find('afrobeat') != -1)

In [ ]:
#number of track_genre = afrobeat 
filter_by_value.count()

1000

In [ ]:
filter_by_value.take(2)

['1000,4pR4oQJULf7FDB54TleMyH,Limoblaze;Lecrae;Happi,Jireh (My Provider),Jireh (My Provider),64,168000,FALSE,0.443,0.778,0,-7.564,0,0.266,0.241,0,0.215,0.628,128.25,5,afrobeat',
 '1001,4Ds4bq6aqOSAp1T7DikOi4,Criolo,Ainda Há Tempo,Ainda Há Tempo,44,318360,FALSE,0.58,0.8,6,-7.528,1,0.345,0.281,0,0.0708,0.483,81.303,4,afrobeat']

In [ ]:
#to save results 
filter_by_value.saveAsTextFile("filter_afrobeat5.txt")

In [ ]:
#group all track_genre = afrobeat 
group_result_filter = filter_by_value.groupBy(lambda a: a.find('afrobeat') != -1).mapValues(list)

In [ ]:
#save result of grouping
group_result_filter.saveAsTextFile("result_group_by_afrobeat6.txt")